# Maximise Prime Item Inventory

Amazon wants to maximise the storage capacity of its 500,000 square-foot warehouse by prioritising a specific batch of prime items. The specific prime product batch detailed in the `inventory` table must be maintained.

So, if the prime product batch specified in the `item_category` column included 1 laptop & 1 side table, that would be the base bach. We could not add another laptop without also adding a side table; they come all together as a batch set.

After prioritising the maximum number of prime batches, any remaining square footage will be utilised to stock non-prime batches, which also come in batch sets & cannot be separated into individual items.

Write a query to find the maximum number of prime & non-prime batches that can be stored in the 500,000 square feet warehouse based on the following criteria:

* Prioritise stocking prime batches
* After accommodating prime items, allocate any remaining space to non-prime batches

Output the `item_type` with `prime_eligible` first followed by `not_prime`, along with the maximum number of batches that can be stocked.

Assumptions:

* Again, products must be stocked in batches, so we want to find the largest available quantity of prime batches, & then the largest available quantity of non-rime batches
* Non-prime items must always be available in stock to meet customer demand, so the non-prime item count should never be zero.
* Item count should be whole numbers (integers).

# Answer

I am using the `inventory` table:

```
CREATE TABLE inventory (
	item_id smallint,
	item_type text,
	item_category text,
	square_footage numeric(5, 2)
);

COPY inventory
FROM '/YourDirectory/inventory.csv'
WITH (FORMAT CSV, HEADER);

SELECT * FROM inventory;
```

<img src = "inventory Table.png" width = "600" style = "margin:auto"/>

Ok, so if I'm understanding correctly, if I have 1 of each prime item in the `inventory` table, that is considered 1 prime batch... Same for non-prime items -- if I have 1 of each non-prime item, that is considered 1 non-prime batch... But then each single batch will have the an item count. We then just prioritise the prime batch over the non-prime batch to maximise 500,000 square feet. 

This one's a tough one... I'll handle the prime vs non-prime items separately. This is the query for prime-eligible batches.

```
SELECT item_type,
	   sum(square_footage) AS sq_ft,
	   count(*) AS items_per_batch,
	   div(500000, sum(square_footage)) AS batches,
	   div(500000, sum(square_footage)) * count(*)
	   	   AS total_items,
	   sum(square_footage) * div(500000, 
	   	   sum(square_footage)) AS sq_ft_used
FROM inventory
GROUP BY item_type
HAVING item_type = 'prime_eligible';
```

<img src = "Prime Eligible Batch.png" width = "600" style = "margin:auto"/>

With a maximum of 900 prime-eligible batches stored in the warehouse -- that's 900 batchs with 6 items each, totaling to 5400 items for prime-eligible batches.

Now for non-prime batches.

```
SELECT item_type,
	   sum(square_footage) AS sq_ft,
	   count(*) AS items_per_batch,
	   floor((500000 - (SELECT sum(square_footage) * 
	   	   div(500000, sum(square_footage)) 
		   FROM inventory GROUP BY item_type 
		   HAVING item_type = 'prime_eligible')) / 
		   sum(square_footage)) AS batches,
	   floor((500000 - (SELECT sum(square_footage) * 
	   	   div(500000, sum(square_footage)) 
		   FROM inventory GROUP BY item_type 
		   HAVING item_type = 'prime_eligible')) / 
		   sum(square_footage)) * count(*) AS total_items,
	   floor((500000 - (SELECT sum(square_footage) * 
	   	   div(500000, sum(square_footage)) 
		   FROM inventory GROUP BY item_type 
		   HAVING item_type = 'prime_eligible')) / 
		   sum(square_footage)) * sum(square_footage)
		   AS sq_ft_used
FROM inventory
GROUP BY item_type
HAVING item_type = 'not_prime';
```

<img src = "Non-Prime Batch.png" width = "600" style = "margin:auto"/>

I subtracted the `sq_ft_used` calculation (from the query for the prime-eligible batches) from 500,000 to find the available space for non-prime batches, which is 320 square feet. Only two non-prime batches can fit into the available space, & with 4 items per batch, that totals 8 items for non-prime batches.

Now we can perform a union on the two queries.

```
(SELECT item_type,
	   sum(square_footage) AS sq_ft,
	   count(*) AS items_per_batch,
	   div(500000, sum(square_footage)) AS batches,
	   div(500000, sum(square_footage)) * count(*)
	   	   AS total_items,
	   sum(square_footage) * div(500000, 
	   	   sum(square_footage)) AS sq_ft_used
FROM inventory
GROUP BY item_type
HAVING item_type = 'prime_eligible')
UNION
(SELECT item_type,
	   sum(square_footage) AS sq_ft,
	   count(*) AS items_per_batch,
	   floor((500000 - (SELECT sum(square_footage) * 
	   	   div(500000, sum(square_footage)) 
		   FROM inventory GROUP BY item_type 
		   HAVING item_type = 'prime_eligible')) / 
		   sum(square_footage)) AS batches,
	   floor((500000 - (SELECT sum(square_footage) * 
	   	   div(500000, sum(square_footage)) 
		   FROM inventory GROUP BY item_type 
		   HAVING item_type = 'prime_eligible')) / 
		   sum(square_footage)) * count(*) AS total_items,
	   floor((500000 - (SELECT sum(square_footage) * 
	   	   div(500000, sum(square_footage)) 
		   FROM inventory GROUP BY item_type 
		   HAVING item_type = 'prime_eligible')) / 
		   sum(square_footage)) * sum(square_footage)
		   AS sq_ft_used
FROM inventory
GROUP BY item_type
HAVING item_type = 'not_prime')
ORDER BY item_type DESC;
```

<img src = "Maximising Inventory.png" width = "600" style = "margin:auto"/>

To maximise inventory of prime-eligible items, this warehouse can fit 5400 prime-eligible items & 8 non-prime items, totaling to 5408 items. Prime-eligible batches take up 499,680 square feet & non-prime items take up 257 square feet, occupying a total of 499,937 square feet.